In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# Install Libs

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install flask-ngrok

In [4]:
!pip install pyngrok

     |████████████████████████████████| 745 kB 4.3 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=c359ee7f1ad5a75061a06cb29da65931a7159659c8d254ffbc513ac44fb4a371
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [5]:
!ngrok authtoken 23cmJrQr0V6QY3f4Soj65SVLSpW_4FJgTG1Rf3apHrvUoN2hm

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [6]:
# Import neccessary package
!pip install Polygon3

  Created wheel for Polygon3: filename=Polygon3-3.0.9.1-cp37-cp37m-linux_x86_64.whl size=102633 sha256=1c7d3a398d5d1171a5c01918898c774755ca8e047bac8a3d965170e17ae942ee
  Stored in directory: /root/.cache/pip/wheels/0d/f3/a1/d9909fbad83c438786f3fbde79b0636c9e843107bad74baba7
Successfully built Polygon3


In [7]:
!pip install vietocr==0.3.5

     |████████████████████████████████| 61 kB 3.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 948 kB 8.7 MB/s 
     |████████████████████████████████| 280 kB 70.8 MB/s 
  Created wheel for gdown: filename=gdown-3.11.0-py3-none-any.whl size=9633 sha256=d90e1634ae6ef7cde6b100f5d1468644116ec6be71164b18ed02565b359511ca
  Stored in directory: /root/.cache/pip/wheels/05/e6/10/9cbfea8dcf9fde0f406da1e4c71d5c3cf3c99e0502d7f08ac6
Successfully built gdown
  Attempting uninstall: lmdb
    Found existing installation: lmdb 0.99
    Uninstalling lmdb-0.99:
      Successfully uninstalled lmdb-0.99
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Attempting uninstall: gdown
    Found existing installation: gdown 3.6.4
    Uninstalling gdown-3.6.4:
      Successfully uninst

# App

In [8]:
import os
import shutil
import cv2
import random
from datetime import datetime
from PIL import Image
from flask import Flask
from flask import Flask, render_template, request, flash, url_for, send_file, jsonify, redirect
from flask_ngrok import run_with_ngrok

static_folder="/content/drive/MyDrive/Project_Deeplearning/demo/static"
template_folder="/content/drive/MyDrive/Project_Deeplearning/demo/templates"

pretrained_craft_1 = "/content/drive/MyDrive/Project_Deeplearning/text_detection/model/craft/2022_01_12_06_13_lr_0.00001/26_35.727.pth"
pretrained_craft_2 = "/content/drive/MyDrive/Project_Deeplearning/text_detection/model/craft/2022_01_11_01_20_lr_0.0001/19_46.025.pth"
pretrained_linkrefine = "/content/drive/MyDrive/Project_Deeplearning/text_detection/model/pretrained/craft_refiner_CTW1500.pth"

In [9]:
! echo $PYTHONPATH
import os
os.environ['PYTHONPATH'] = "/content/drive/MyDrive/Project_Deeplearning;/content/drive/MyDrive/Project_Deeplearning/text_detection"

! echo $PYTHONPATH

%cd "/content/drive/MyDrive/Project_Deeplearning"

/env/python
/content/drive/MyDrive/Project_Deeplearning;/content/drive/MyDrive/Project_Deeplearning/text_detection
/content/drive/MyDrive/Project_Deeplearning


In [26]:
def crop_image_by_bbox(image, box):
        """
        Crop image by box, after cropping we have image with shape maxWidth, maxHeight
        :param image:
        :param box:
        :return:
        """
        # Convert from PIL Image to cv2
        box = np.array(box)
        print("Box: ", box)

        (tl, tr, br, bl) = box
        print("TL: ", tl)
        print("tr: ", tr)
        print("br: ", br)
        print("bl: ", bl)

        widthA = np.sqrt(((br[0] - bl[0]) * 2) + ((br[1] - bl[1]) * 2))
        widthB = np.sqrt(((tr[0] - tl[0]) * 2) + ((tr[1] - tl[1]) * 2))

        # ...and now for the height of our new image
        heightA = np.sqrt(((tr[0] - br[0]) * 2) + ((tr[1] - br[1]) * 2))
        heightB = np.sqrt(((tl[0] - bl[0]) * 2) + ((tl[1] - bl[1]) * 2))

        # take the maximum of the width and height values to reach
        # our final dimensions
        maxWidth = max(int(widthA), int(widthB))
        maxHeight = max(int(heightA), int(heightB))
        width = maxWidth
        height = maxHeight
        dst = np.array([
                [0, 0],
                [maxWidth - 1, 0],
                [maxWidth - 1, maxHeight - 1],
                [0, maxHeight - 1]], dtype="float32")

        # calculate the perspective transform matrix and warp
        # the perspective to grab the screen
        M = cv2.getPerspectiveTransform(np.array(box, dtype=np.float32), dst)
        warp = cv2.warpPerspective(cv_image, M, (maxWidth, maxHeight))
        warp_pil = Image.fromarray(cv2.cvtColor(warp, cv2.COLOR_BGR2RGB))
        return warp_pil, M

In [ ]:
# from vietocr.tool.config import Cfg
# from vietocr.tool.predictor import Predictor
# from glob import glob
# from google.colab.patches import cv2_imshow
# import torch

# config = Cfg.load_config_from_name('vgg_transformer')
# config['device'] = 'cuda' if torch.cuda.is_available() else 'cpu'
# config['predictor']['beamsearch'] = False
# detector = Predictor(config)

# def get_text(image, bbox):
#     img_box, M = crop_image_by_bbox(image, bbox)
#     pred = detector.predict(img_box)
#     print(pred)
#     return pred

# # gt_path = -1
# # img_path = -1
# # for path in glob(os.path.join("/content/drive/MyDrive/BK_AI/Dataset/VinText/gt_test", "*.txt")):
# #     gt_path = path
# #     break
# # img_name = os.path.basename(gt_path).replace("gt_", "").replace(".txt", ".jpg")
# # img_path = os.path.join("/content/drive/MyDrive/BK_AI/Dataset/VinText/unseen_test_images", img_name)
# image = cv2.imread("/content/drive/MyDrive/im0004.jpg")
# cv2_imshow(image)

# bbox = [[ 48, 40], 
#  [276, 49],
#  [275, 73],
#  [ 47, 63]]
# pred = get_text(image, bbox)

In [31]:
# from text_detection.inferences import test_net
from text_detection.gaussianMap import imgproc
from text_detection.net.craft import CRAFT
from text_detection.net.refinenets import RefineNet
from text_detection.net.torch_util import copyStateDict
from text_detection import utils

from google.colab.patches import cv2_imshow
from torch.autograd import Variable

import torch
import cv2
import numpy as np
import copy
import time

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Load craft pretrained model
craft = CRAFT()
print('Loading weights from checkpoint (' + pretrained_craft_2 + ')')
craft.load_state_dict(copyStateDict(torch.load(pretrained_craft_2, map_location=device)))

# Load LinkRefiner
refine_net = RefineNet()

print('Loading weights of refiner from checkpoint (' + pretrained_linkrefine + ')')
refine_net.load_state_dict(copyStateDict(torch.load(pretrained_linkrefine, map_location=device)))

def test_net(image, net, device, refine_net=None, poly=False, text_threshold=0.7, link_threshold=0.4, low_text=0.4, canvas_size=1280, mag_ratio=1.5):
    net = net.to(device)
    net.eval()

    t0 = time.time()
    # resize image
    img_resized, target_ratio, size_heatmap = imgproc.resize_aspect_ratio(image, canvas_size,
                                                                          interpolation=cv2.INTER_LINEAR,
                                                                          mag_ratio=mag_ratio)
    ratio_h = ratio_w = 1 / target_ratio

    # preprocessing
    x = imgproc.normalizeMeanVariance(img_resized)
    x = torch.from_numpy(x).permute(2, 0, 1)  # [h, w, c] to [c, h, w]
    x = Variable(x.unsqueeze(0))  # [c, h, w] to [b, c, h, w]
    x = x.to(device)

    # forward pass
    print("Device1: ", device)
    with torch.no_grad():
        y, feature = net(x)

    print("Device: ", device)

    # make score and link map
    score_text = y[0, :, :, 0].cpu().data.numpy()
    score_link = y[0, :, :, 1].cpu().data.numpy()
    # score_text = y[0, :, :, 0]
    # score_link = y[0, :, :, 1]

    # refine link
    if refine_net is not None:
        print('co dung refineeeeeeeeee nheeeeeeeeeeee')
        refine_net = refine_net.to(device)
        refine_net.eval()
        with torch.no_grad():
            y_refiner = refine_net(y, feature)
        score_link = y_refiner[0, :, :, 0].cpu().data.numpy()

    t0 = time.time() - t0
    t1 = time.time()

    # Post-processing
    boxes, polys = utils.getDetBoxes(score_text, score_link, text_threshold, link_threshold, low_text, poly)

    # coordinate adjustment
    boxes = utils.adjustResultCoordinates(boxes, ratio_w, ratio_h)
    polys = utils.adjustResultCoordinates(polys, ratio_w, ratio_h)
    for k in range(len(polys)):
        if polys[k] is None: polys[k] = boxes[k]

    t1 = time.time() - t1

    # render results (optional)
    render_img = score_text.copy()
    render_img = np.hstack((render_img, score_link))
    ret_score_text = imgproc.cvt2HeatmapImg(render_img)

    # if args.show_time:
        # print("\ninfer/postproc time : {:.3f}/{:.3f}".format(t0, t1))
    return boxes, polys, ret_score_text


def detect(file_name, use_refine=False):
    global device, craft, refine_net
    # Load image
    file_path = os.path.join(static_folder, 'raw', file_name)
    image = imgproc.loadImage(file_path)

    # link_refiner = None
    # poly = False

    if use_refine:
        bboxes, polys, score_text = test_net(image, craft, device, refine_net, poly=True)
    else:
        bboxes, polys, score_text = test_net(image, craft, device, refine_net=None, poly=False)

    # print(polys)
    return bboxes, polys, score_text


def predict(file_name, use_refine=False):
    image = cv2.imread(os.path.join(static_folder, 'raw', file_name))
    height, width, channel = image.shape

    font = cv2.FONT_HERSHEY_DUPLEX
    recog_result = list()

    ##### TEXT DETECTION
    _, polys, _ = detect(file_name, use_refine)

    # Draw to image:
    img_cp = image.copy()
    for i, poly in enumerate(polys):
        box = np.array(poly, dtype=np.int32)
        img_cp = cv2.polylines(img_cp, [box], isClosed=1, color=(0, 255, 0), thickness=2)
        img_cp = cv2.putText(img_cp, str(i + 1), (box[0][0], box[0][1]), font, 1, (0, 0, 255), 1)

        # With each box, try to crop word image, and go through text recognition
        #####
        ##    TEXT RECOGNITION HERE
        #####
        # print('Box: ', box)
        # recog_result.append((str(i + 1), get_text(image, box)))
    now = datetime.now()
    dt_string = now.strftime("%Y%m%d%H%M%S")
    name, extension = file_name.rsplit('.', 1)
    save_file_name = 'result/%s.%s'%(name + dt_string, extension)
    cv2.imwrite(os.path.join(static_folder, save_file_name), img_cp)
    return save_file_name, recog_result

# predict("Exercise No. 5.png", use_refine=True)

Loading weights from checkpoint (/content/drive/MyDrive/Project_Deeplearning/text_detection/model/craft/2022_01_11_01_20_lr_0.0001/19_46.025.pth)
Loading weights of refiner from checkpoint (/content/drive/MyDrive/Project_Deeplearning/text_detection/model/pretrained/craft_refiner_CTW1500.pth)


In [ ]:
# cua minh
# !ngrok authtoken 23cmJrQr0V6QY3f4Soj65SVLSpW_4FJgTG1Rf3apHrvUoN2hm

In [ ]:
# cua Phuc
# !ngrok authtoken 23dqPkOzyjUbUKS1siT68aoOenw_2VfFhYQHFEM9Whgn8rAAB

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
# AS
# !ngrok authtoken 23eDsNU4TAlQkRDWE1o0fFuA0GZ_3BNvFVuewwuymgnQby9mY

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [17]:
# 01
!ngrok authtoken 23eQ7q3vBeHsyFg2hhXupA8H7jt_4GSGtU24Vv7XkJg9aXzPa

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [32]:
app = Flask(__name__, static_folder=static_folder, template_folder=template_folder)
run_with_ngrok(app)
@app.route("/", methods=["GET", "POST"])
def home():
    if request.method == "POST":
        file = request.files['file']
        fn = file.filename
        file.save(os.path.join(static_folder, 'raw', fn))
        if request.form['submit_button'] == 'refine':
            sfn, result = predict(fn, use_refine=True)
        if request.form['submit_button'] == 'non-refine':
            sfn, result = predict(fn, use_refine=False)
        return render_template('result.html', fn=sfn, result=result)
    return render_template("home.html")

app.run()